In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [14]:
csopi = pd.read_excel("csoportNOTnorm.xlsx") #Boltok

In [15]:
csopi.head()

,EGYÉB CIKKEK,FRISSÁRU,HÚS,HÚSKÉSZTMÉNYEK,ITALOK,NON FOOD TERMÉKEK,"PAPÍR, VEGYIÁRU",SZOLGÁLTATÁSOK,"ZÖLDSÉG, GYÜMÖLCS",ÉLELMISZER
30103,16600,26900,7400,1300,67800,9000,40300,24000,8000,241800
30104,47300,66800,34700,8200,237500,42600,160900,29200,36000,456100
30105,31700,6600,2700,0,49400,5500,22100,13400,3300,487300
30106,3300,5300,0,0,13800,600,19500,10800,400,34000
30108,6000,21200,1600,600,18800,400,21500,9500,2700,168100


In [19]:
csopi["sum"]=csopi.sum(axis=1)
for i in csopi.index.values.tolist():
    csopi.loc[[i]] = csopi.loc[[i]]/csopi.loc[[i]]["sum"].max()
    
csopi

,EGYÉB CIKKEK,FRISSÁRU,HÚS,HÚSKÉSZTMÉNYEK,ITALOK,NON FOOD TERMÉKEK,"PAPÍR, VEGYIÁRU",SZOLGÁLTATÁSOK,"ZÖLDSÉG, GYÜMÖLCS",ÉLELMISZER,sum
30103,0.037463,0.060709,0.016701,0.002934,0.153013,0.020311,0.090950,0.054164,0.018055,0.545701,1
30104,0.042259,0.059680,0.031002,0.007326,0.212186,0.038060,0.143751,0.026088,0.032163,0.407487,1
30105,0.050965,0.010611,0.004341,0.000000,0.079421,0.008842,0.035531,0.021543,0.005305,0.783441,1
30106,0.037628,0.060433,0.000000,0.000000,0.157355,0.006842,0.222349,0.123147,0.004561,0.387685,1
30108,0.023962,0.084665,0.006390,0.002396,0.075080,0.001597,0.085863,0.037939,0.010783,0.671326,1
30109,0.124294,0.048023,0.013183,0.000000,0.130885,0.098870,0.268362,0.073446,0.006591,0.236347,1
30110,0.057767,0.071429,0.019516,0.000000,0.180718,0.007026,0.146370,0.062061,0.051913,0.403201,1
30111,0.010350,0.118130,0.017131,0.008565,0.180228,0.023911,0.118130,0.089222,0.030335,0.403997,1
30113,-0.003623,0.259964,0.000000,0.009964,0.552536,0.135870,0.446558,0.086957,0.029891,-0.518116,1
30116,0.016020,0.083179,0.062230,0.004929,0.242760,-0.003697,0.180530,0.128158,0.043746,0.242144,1


In [25]:
len(csopi)

30

In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.cross_validation as xval
import sklearn.datasets as skd
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.metrics import roc_curve as roc
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score as auc_score
from sklearn.metrics import accuracy_score as acc
%matplotlib inline

m = knn(5)
m.fit(csopi.drop("ITALOK", axis=1), csopi["ITALOK"])
pred = m.predict(csopi.drop("ITALOK", axis=1))
prob = m.predict_proba(csopi.drop("ITALOK", axis=1))[:,1]


plt.show()

In [56]:
pred

array([ 0.13023679,  0.15419095,  0.06911134,  0.09308511,  0.06911134,
        0.09308511,  0.18022841,  0.15811567,  0.04432133,  0.15735462,
        0.09308511,  0.18022841,  0.14839572,  0.13023679,  0.09308511,
        0.18022841,  0.13023679,  0.14839572,  0.04432133,  0.06911134,
        0.13088512,  0.15419095,  0.10612492,  0.03520115,  0.12747253,
        0.14839572,  0.13023679,  0.15419095,  0.10612492,  0.06911134])